In [1]:
import torch
from torch.nn import functional as F

In [2]:
import NanoGPTLangugageModel as nano
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
model = nano.NanoGPTLanguageModel()
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(nano.device)
encode, decode = nano.encode, nano.decode

In [6]:
start_str = "\n"
idx = torch.tensor(encode(start_str), dtype=torch.long, device=device).unsqueeze(0)
print(decode(m.generate(idx = idx, max_new_tokens=256)[0].tolist()))


Togerous before them, after would those thrifts;
Yield these have I look not how they there is no.
Come, let's upon the desire
Be count in affection. My noble looking throw
Grows all.

Lord:
We thank that Paris to him.

FLORIZEL:
For child, be but long.

P


In [80]:
import torch
from torch.nn import functional as F
from einops import rearrange, einsum

In [81]:
# matrix mult using einsum w/ B H T D tensor
torch.manual_seed(1337)
B, H, T, D = 1, 3, 4, 3
out = torch.randint(-10, 10, (B, H, T, D)) # (B, H, T, D)
out_proj = torch.randint(-10, 10, (H, D, H*D)) # (H, D, C)

In [83]:
out_proj_T = rearrange(out_proj, 'h d c -> c h d') # this isn't transpose, how to transpose
einsum(out, out_proj_T, 'b h t d, c h d -> b t c') # (B, H, T, C)

tensor([[[-154,  224,  175,   75,  -45,   15,  224,  -18, -100],
         [-118,   61,   -8,  180,  -28,   -3,  -33,  -33,  -73],
         [ 137, -124,  -38,  -95,  -55,   49, -115,  113,   70],
         [  97,    7, -171,   -8,   91,   64,  123, -133, -206]]])

In [87]:
new_out = rearrange(out, 'b h t d -> b t (h d)') # (B, T, C)
new_out_proj = rearrange(out_proj, 'h d c -> (h d) c') # (C, C)
new_out_proj_T = new_out_proj.T

expected = F.linear(new_out, new_out_proj, bias=None) # (B, T, C)
actual = einsum(new_out, new_out_proj_T, 'b t c1, c1 c2  -> b t c2') # (B, T, C)
actual, expected == actual

(tensor([[[  49,  -97,   53,   34, -107, -146,  158,  -85,  -36],
          [ -72,  200,  -70,  175, -162, -143, -112, -187,    6],
          [  91,   55,  -38,   38,   30,   39,  -87,   61,   -4],
          [  50,   18,   86, -165,   72,  210,  146,  -25, -115]]]),
 tensor([[[True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True],
          [True, True, True, True, True, True, True, True, True]]]))

In [88]:
C = H * D
w_in = torch.randint(-10, 10, (C, 4*C)) 
w_out = torch.randint(-10, 10, (4*C, C))
w_in, w_out

(tensor([[ -1,  -2,  -7,  -4,   9,  -5,  -8,   4,  -9,   5,  -7,   0,   3,  -6,
            4, -10,  -3,   8,  -3,  -5,   6,  -6, -10,  -6,  -3,  -8,   7,  -2,
            6,  -7,  -2,  -6,   4,  -4,  -3,  -8],
         [  4,   9,  -9,  -7, -10,   4,   4,   7,  -8,  -7,  -4,   4,   7,   6,
           -4,   7,   0,  -9,  -1,   2,  -5,   7,   8,  -6,  -9,  -1,   6,  -9,
            8,   0,  -9,   0,  -4,   9,   1,   5],
         [  9,  -7,   1,  -1,  -2,   9, -10,   8,  -8,   2,   3,   5,  -8,  -1,
           -9,   3,   0,  -7,   3,   6,  -6,  -3,  -3,   1,   5,  -8,   2,   5,
           -3,  -3,  -1, -10,   7,   0,  -7,  -4],
         [ -6,  -3,  -8,   2,  -2,   5,   3,  -2,  -2,   2,  -5,  -3,   0,  -3,
           -2,  -2,  -5,  -1,   4,   3,  -6,   3,   5, -10,   8,   1,   6,   8,
           -9,  -9,   1,   2,  -3,  -2,   1,  -4],
         [ -4,   5,  -6,  -9,   7,  -1,  -3,  -6,  -6,  -1,  -8,   3,   7,   1,
            6,   4,   7,  -8,  -3,  -7,   9,   9,   4,   3,  -8,  -9,   7,  

In [89]:
expected_mlp = F.linear(actual, w_in.T, bias=None) # (B, T, C)
actual_mlp = einsum(actual, w_in, 'b t c1, c1 c2  -> b t c2') # (B, T, C)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x9 and 36x9)